## Import relevant libraries

In [1]:
import pandas as pd
import numpy as np
import text_normalizer as tn

np.set_printoptions(precision=2, linewidth=80)

## Load and normalize data

In [2]:
datasets = pd.read_csv('Movie_review.csv')
reviews = np.array(datasets['review'])
sentiments = np.array(datasets['sentiment'])

# extract data for model evaluation
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]
sample_review_ids = [7626, 3533, 13010]

# normalize dataset
norm_test_reviews = tn.normalize_corpus(test_reviews)

## Sentiment Analysis with AFINN

In [3]:
from afinn import Afinn
afn = Afinn(emoticons=True)

## Predict sentiment for sample reviews

In [4]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('Review :', review)
    print('Actual Sentiment:', sentiment)
    print('Predicted Sentiment Polarity:',afn.score(review))
    print('--'*50)

Review : no comment - stupid movie, acting average or worse... screenplay - no sense at all... SKIP IT!
Actual Sentiment: negative
Predicted Sentiment Polarity: -7.0
----------------------------------------------------------------------------------------------------
Review : I don't care if some people voted this movie to be bad. If you want the Truth this is a Very Good Movie! It has every thing a movie should have. You really should Get this one.
Actual Sentiment: positive
Predicted Sentiment Polarity: 3.0
----------------------------------------------------------------------------------------------------
Review : Worst horror film ever but funniest film ever rolled in one you have got to see this film it is so cheap it is unbeliaveble but you have to see it really!!!! P.s watch the carrot
Actual Sentiment: positive
Predicted Sentiment Polarity: -3.0
----------------------------------------------------------------------------------------------------


## Predict sentiment for test dataset

In [5]:
sentiment_polarity = [afn.score(review) for review in test_reviews]
predicted_sentiments = ['positive' if pl >= 1.0 else 'negative' for pl in sentiment_polarity]

## Evaluate model performance

In [7]:
from model_evaluation_utils import display_model_performance_metrics

In [8]:
display_model_performance_metrics(test_sentiments,predicted_sentiments,classes=['positive','negative'])


Model Performance metrics:
------------------------------
Accuracy : 71.18%
Precision : 66.66%
Recall : 84.90%
F1 Score : 74.68%

Model Classification reports:
------------------------------
              precision    recall  f1-score   support

    negative       0.79      0.57      0.67      7490
    positive       0.67      0.85      0.75      7510

    accuracy                           0.71     15000
   macro avg       0.73      0.71      0.71     15000
weighted avg       0.73      0.71      0.71     15000

Prediction Confusion Matrix : 
------------------------------
                 Predicted:         
                   positive negative
Actual: positive       6376     1134
        negative       3189     4301


## Sentiment Analysis with SentiWordNet

In [9]:
from nltk.corpus import sentiwordnet as swn

awesome = list(swn.senti_synsets('awesome','a'))[0]

In [10]:
print('Positive Polarity Score:', awesome.pos_score())
print('Negative Polarity Score:', awesome.neg_score())
print('Objective Score:', awesome.obj_score())

Positive Polarity Score: 0.875
Negative Polarity Score: 0.125
Objective Score: 0.0


## Build model

In [11]:
def analyze_sentiment_sentiwordnet_lexicon(review, verbose=False):
    # tokenize and POS tag text words
    tagged_text = [(token.text, token.tag_) for token in tn.nlp(review)]
    pos_score = neg_score = token_count = obj_score = 0
    # get wordnet synsets based on POS tag
    # get sentiment scores if synsets are found
    for word, tag in tagged_text:
        ss_set = None
        if 'NN' in tag and list(swn.senti_synsets(word,'n')):
            ss_set = list(swn.senti_synsets(word,'n'))[0]
        elif 'VB' in tag and list(swn.senti_synsets(word,'v')):
            ss_set = list(swn.senti_synsets(word,'v'))[0]
        elif 'JJ' in tag and list(swn.senti_synsets(word,'a')):
            ss_set = list(swn.senti_synsets(word,'a'))[0]
        elif 'RB' in tag and list(swn.senti_synsets(word,'r')):
            ss_set = list(swn.senti_synsets(word,'r'))[0]
        # if senti-synsets ids found
        if ss_set:
            # add scores for all found synsets
            pos_score += ss_set.pos_score()
            neg_score += ss_set.neg_score()
            obj_score += ss_set.obj_score()
            token_count += 1
    # aggregate final scores
    final_score = pos_score-neg_score
    norm_final_score = round(float(final_score)/token_count, 2)
    final_sentiment = 'positive' if norm_final_score>=0 else 'negative'
    if verbose:
        norm_obj_score = round(float(obj_score) / token_count, 2)
        norm_pos_score = round(float(pos_score) / token_count, 2)
        norm_neg_score = round(float(neg_score) / token_count, 2)
        # to display results in a nice table
        sentiment_frame = pd.DataFrame([[final_sentiment, norm_obj_score, norm_pos_score, 
                                         norm_neg_score, norm_final_score]],
                                       columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                             ['Predicted Sentiment', 'Objectivity',
                                                              'Positive', 'Negative', 'Overall']], 
                                                             codes=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
    return final_sentiment

## Predict sentiment for sample reviews

In [12]:
for review, sentiment in zip(test_reviews[sample_review_ids],test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    pred = analyze_sentiment_sentiwordnet_lexicon(review, verbose=True)    
    print('-'*60)

REVIEW: no comment - stupid movie, acting average or worse... screenplay - no sense at all... SKIP IT!
Actual Sentiment: negative
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            negative        0.74      0.1     0.17   -0.07
------------------------------------------------------------
REVIEW: I don't care if some people voted this movie to be bad. If you want the Truth this is a Very Good Movie! It has every thing a movie should have. You really should Get this one.
Actual Sentiment: positive
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.74      0.2     0.06    0.14
------------------------------------------------------------
REVIEW: Worst horror film ever but funniest film ever rolled in one you have got to see this film it is so cheap it is unbeliaveble but you have to see it really!!!! P.s watch 

In [13]:
predicted_sentiments = [analyze_sentiment_sentiwordnet_lexicon(review, verbose=False) for review in norm_test_reviews]

## Evaluate model performance

In [14]:
display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=predicted_sentiments, 
                                  classes=['positive', 'negative'])


Model Performance metrics:
------------------------------
Accuracy : 68.43%
Precision : 66.48%
Recall : 74.51%
F1 Score : 70.27%

Model Classification reports:
------------------------------
              precision    recall  f1-score   support

    negative       0.71      0.62      0.66      7490
    positive       0.66      0.75      0.70      7510

    accuracy                           0.68     15000
   macro avg       0.69      0.68      0.68     15000
weighted avg       0.69      0.68      0.68     15000

Prediction Confusion Matrix : 
------------------------------
                 Predicted:         
                   positive negative
Actual: positive       5596     1914
        negative       2822     4668


## Sentiment Analysis with VADER

In [15]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Build model

In [16]:
def analyze_sentiment_vader_lexicon(review, 
                                    threshold=0.1,
                                    verbose=False):
    # pre-process text
    review = tn.strip_html_tags(review)
    review = tn.remove_accented_chars(review)
    review = tn.expand_contractions(review)
    
    # analyze the sentiment for review
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    # get aggregate scores and final sentiment
    agg_score = scores['compound']
    final_sentiment = 'positive' if agg_score >= threshold\
                                   else 'negative'
    if verbose:
        # display detailed sentiment statistics
        positive = str(round(scores['pos'], 2)*100)+'%'
        final = round(agg_score, 2)
        negative = str(round(scores['neg'], 2)*100)+'%'
        neutral = str(round(scores['neu'], 2)*100)+'%'
        sentiment_frame = pd.DataFrame([[final_sentiment, final, positive,
                                        negative, neutral]],
                                        columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                                      ['Predicted Sentiment', 'Polarity Score',
                                                                       'Positive', 'Negative', 'Neutral']], 
                                                              codes=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
    
    return final_sentiment

## Predict sentiment for sample reviews

In [17]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    pred = analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=True)    
    print('-'*60)

REVIEW: no comment - stupid movie, acting average or worse... screenplay - no sense at all... SKIP IT!
Actual Sentiment: negative
     SENTIMENT STATS:                                         
  Predicted Sentiment Polarity Score Positive Negative Neutral
0            negative           -0.8     0.0%    40.0%   60.0%
------------------------------------------------------------
REVIEW: I don't care if some people voted this movie to be bad. If you want the Truth this is a Very Good Movie! It has every thing a movie should have. You really should Get this one.
Actual Sentiment: positive
     SENTIMENT STATS:                                                     
  Predicted Sentiment Polarity Score Positive             Negative Neutral
0            negative          -0.16    16.0%  14.000000000000002%   69.0%
------------------------------------------------------------
REVIEW: Worst horror film ever but funniest film ever rolled in one you have got to see this film it is so cheap it is unb

## Predict sentiment for test dataset

In [18]:
predicted_sentiments = [analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=False) for review in test_reviews]

## Evaluate model performance

In [19]:
display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=predicted_sentiments, 
                                  classes=['positive', 'negative'])


Model Performance metrics:
------------------------------
Accuracy : 71.07%
Precision : 67.02%
Recall : 83.14%
F1 Score : 74.21%

Model Classification reports:
------------------------------
              precision    recall  f1-score   support

    negative       0.78      0.59      0.67      7490
    positive       0.67      0.83      0.74      7510

    accuracy                           0.71     15000
   macro avg       0.72      0.71      0.71     15000
weighted avg       0.72      0.71      0.71     15000

Prediction Confusion Matrix : 
------------------------------
                 Predicted:         
                   positive negative
Actual: positive       6244     1266
        negative       3073     4417
